In [2]:
import gym

In [2]:
print(gym.__version__)

0.23.1


In [3]:
env = gym.make("ALE/SpaceInvaders-v5", render_mode = 'human')

A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]


In [8]:
episodes = 10


for episode in range (1, episodes):
    state = env.reset() # restart the simulation
    done = False
    score = 0
    
    
    
    while not done:
        #env.render(mode = 'human')
        state, reward, done, info = env.step(env.action_space.sample())
        score += reward
        
    print(f'Episode: {episode}  Score: {score}')
    
env.close()
    

Episode: 1  Score: 50.0
Episode: 2  Score: 110.0
Episode: 3  Score: 35.0
Episode: 4  Score: 250.0
Episode: 5  Score: 20.0
Episode: 6  Score: 120.0
Episode: 7  Score: 110.0
Episode: 8  Score: 120.0
Episode: 9  Score: 35.0


In [4]:
env.close()

In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam

2022-05-22 13:32:06.717353: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sakib/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-05-22 13:32:06.717372: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/sakib/.local/lib/python3.8/site-packages/tensorflow/python/autograph/utils/testing.py:21: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [6]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Conv2D(32, (8,8), strides = (4,4), activation = 'relu', input_shape = (3, height, width, channels)))
    model.add(Conv2D(64, (4, 4), strides = (2,2), activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dense(actions, activation = 'linear'))
    return model

In [7]:
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [18]:
env.action_space.n

6

In [24]:
model = build_model(height, width, channels, actions)

In [20]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy



In [21]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr = 'eps', value_max = 1., value_min = 0.1, value_test = 0.2, nb_steps = 1000)
    memory = SequentialMemory(limit = 2000, window_length = 3)
    dqn = DQNAgent(model = model, 
                   memory = memory, 
                   policy = policy,
                   enable_dueling_network = True, 
                   dueling_type = 'avg',
                   nb_actions = actions, 
                   nb_steps_warmup = 1000
                  )
    return dqn

In [23]:
del model

In [26]:
dqn = build_agent(model, actions)

In [28]:
dqn.compile(Adam(lr = 0.001))

In [29]:
dqn.fit(env, nb_steps = 4000, visualize = False, verbose = 1)

Training for 4000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
 4000/10000 [===========>..................] - ETA: 1:18:23 - reward: 0.2350done, took 3135.813 seconds
